# Plota PPI do arquivo bruto .VOL do RAINBOWN: 
1. https://docs.wradlib.org/en/1.19.1/io.html
2. https://docs.wradlib.org/en/1.19.1/notebooks/fileio/wradlib_radar_formats.html#Gematronik-Rainbow
3. https://docs.wradlib.org/en/1.19.1/notebooks/fileio/wradlib_load_rainbow_example.html

In [23]:
# importa bibliotecas
import wradlib as wrl
import matplotlib.pyplot as pl
import numpy as np
import warnings

warnings.filterwarnings("ignore")
try:
    get_ipython().run_line_magic("matplotlib inline")
except:
    pl.ion()

# Load Rainbow file
file = '2024010300113700dBZ.vol'
rbdict = wrl.io.read_rainbow(file)

# define a elevação
elev = 0

# Get azimuthal data
azi = rbdict["volume"]["scan"]["slice"][elev]["slicedata"]["rayinfo"]["data"]
azidepth = float(rbdict["volume"]["scan"]["slice"][elev]["slicedata"]["rayinfo"]["@depth"])
azirange = float(rbdict["volume"]["scan"]["slice"][elev]["slicedata"]["rayinfo"]["@rays"])
azires = float(rbdict["volume"]["scan"]["slice"][elev]["rangestep"])
azi = (azi * azirange / 2**azidepth) * azires

# Create range array
stoprange = float(rbdict["volume"]["scan"]["slice"][elev]["stoprange"])
rangestep = float(rbdict["volume"]["scan"]["slice"][elev]["rangestep"])
r = np.arange(0, stoprange, rangestep)

# Get reflectivity data
data = rbdict["volume"]["scan"]["slice"][elev]["slicedata"]["rawdata"]["data"]
datadepth = float(rbdict["volume"]["scan"]["slice"][elev]["slicedata"]["rawdata"]["@depth"])
datamin = float(rbdict["volume"]["scan"]["slice"][elev]["slicedata"]["rawdata"]["@min"])
datamax = float(rbdict["volume"]["scan"]["slice"][elev]["slicedata"]["rawdata"]["@max"])
data = datamin + data * (datamax - datamin) / 2**datadepth

# Get annotation data
unit = rbdict["volume"]["scan"]["slice"][elev]["slicedata"]["rawdata"]["@type"]
time = rbdict["volume"]["scan"]["slice"][elev]["slicedata"]["@time"]
date = rbdict["volume"]["scan"]["slice"][elev]["slicedata"]["@date"]
lon = rbdict["volume"]["sensorinfo"]["lon"]
lat = rbdict["volume"]["sensorinfo"]["lat"]
sensortype = rbdict["volume"]["sensorinfo"]["@type"]
sensorname = rbdict["volume"]["sensorinfo"]["@name"]

# Plot data with annotation
fig = pl.figure(figsize=(8, 8))
cgax, pm = wrl.vis.plot_ppi(data, r=r, az=azi, fig=fig, proj="cg")

title = "{0} {1} {2} {3}\n{4}E {5}N".format(sensortype, sensorname, date, time, lon, lat)
caax = cgax.parasites[0]
t = pl.title(title, fontsize=12)
t.set_y(1.1)
cbar = pl.gcf().colorbar(pm, pad=0.075)
caax.set_xlabel("x_range [km]")
caax.set_ylabel("y_range [km]")
pl.text(1.0, 1.05, "azimuth", transform=caax.transAxes, va="bottom", ha="right")
cbar.set_label("reflectivity [" + unit + "]")

RuntimeError: OGR Error: Unsupported SRS

<Figure size 800x800 with 0 Axes>